This lab is binom:

SUN Xu: 22200118

HOU Dan: 22215394

In [ ]:
! pip install sentence_transformers rank_bm25 nltk

# Objectives

In this lab work, you are going to elabotrate a simple semanti search algorithm as it is one of the building block of RAGs.

# Instructions
- Have a tidy straightforward code
- Readability over performances
- Define Functions and comments
- Read the doc
- Send me your submission with the names of the students of your group by next week (18/11)!

# Imports and data

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Data

Upload the data to collab and download the csv file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_qq = pd.read_csv("/content/drive/MyDrive/HD/M2/RAG/qora_qestions.csv") # Change the name of the file if necessary
df_qq.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Initial data analysis

Perform some initial data analysis

In [ ]:
# how many questions are in the dataset?
print(df_qq.shape)
# check for null values
print(f"we find all the null cols , {df_qq.isnull().sum()}")

(404290, 6)
we find all the null cols , id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64


In [ ]:
# 'is_duplicate' distribution?
is_duplicate = df_qq['is_duplicate'].value_counts()
print(is_duplicate)
is_duplicate_perc = is_duplicate / len(df_qq) * 100
print("'is_duplicate' percentage distribution", is_duplicate_perc)

is_duplicate
0    255027
1    149263
Name: count, dtype: int64
'is_duplicate' percentage distribution is_duplicate
0    63.080215
1    36.919785
Name: count, dtype: float64


In [ ]:
# most frequently occurring questions?
print(df_qq['qid1'].value_counts().head(5))
print(df_qq['qid2'].value_counts().head(5))

qid1
8461     50
14110    48
1749     47
20628    47
25984    47
Name: count, dtype: int64
qid2
30782    120
2559     115
4044     100
2561      71
17978     66
Name: count, dtype: int64


From the initial data analysis in the above questions, we can observe that the dataset has a large amount of data, which is why we will subsample the dataset next to keep its size within a manageable range. In addition, the `is_duplicate` distribution is also not average, which can also be made average by subsampling. Finally, identifying high-frequency questions for qid1 and qid2 can also be used to optimize the retrieval model, as it can precache their answers and improve retrieval efficiency.

# Retriever Implementation and Evaluation

## Evaluation method

The dataset is composed of sets of pairs of Qora questionsz with a label indicating if they are to be considered duplicates or not.
An efficient retriever shall retrieve the relevant question out of all the questions.
The retreivers should be evaluated using **at least the Hit Rate and possibly the MAPS.**

## Data structure

In order to evaluate the retrieving methods, you should create the following data sets:
1. A base of unique question with their corresponding ``qid``
2. A key-value dataframe matching ``qid``pairs to a ``is_duplicate`` label

Considering, the number of unique questions (ca 55000) you may subsample the dataset to a manageable size.

## Data preparation code

### Data structures creation

In [ ]:
# create unique question dataframe
df_1 = df_qq[['qid1', 'question1']].rename(columns={'qid1': 'qid', 'question1': 'question'})
df_2 = df_qq[['qid2', 'question2']].rename(columns={'qid2': 'qid', 'question2': 'question'})
df_q = pd.concat((df_1, df_2), axis=0).drop_duplicates()
df_q.sample(5)

,qid,question
205623,308834,How did R2-D2 AND C-3P0 feel about Anakin Skyw...
361727,491594,How much GPA should one maintain in his first ...
89620,150539,What's the most common mistake people make whe...
358775,488318,What tax filing software can I use if I am a U...
345560,473860,Should I learn Scala just for Spark programming?


In [ ]:
# create key-value dataframe
df_labels = df_qq[['qid1', 'qid2', 'is_duplicate']]
df_labels.sample(5)

,qid1,qid2,is_duplicate
276783,395772,18974,0
172663,266547,85595,0
205052,70929,94287,1
166609,258482,258483,0
45262,81131,81132,0



### Data subsampling

Subsample the data so that you have a smaller dataset.
1. Select a set of questions
2. Select the queries as questions from the previous set having at least 1 positive duplicate
3. **Be sure that the ``qid`` of the selected questions are not part of the set of questions that will be indexed in the k-NN for the retriever**

In [ ]:
potential_queries_indices = resample(df_labels.loc[df_labels['is_duplicate'] == 1, 'qid1'].unique(),replace=False, n_samples=1000)
correct_outputs = df_labels.loc[(df_labels['is_duplicate'] == 1)&(df_labels['qid1'].isin(potential_queries_indices)), 'qid2'].unique()
true_positive_basis = set(correct_outputs) - set(potential_queries_indices)
print(len(true_positive_basis))
false_examples = resample(df_q.loc[~df_q['qid'].isin(true_positive_basis), 'qid'].values, n_samples=10000, replace=False)
print(len(false_examples))

all_ids = np.concatenate((list(true_positive_basis), false_examples)) # ids of the chunks for the vdb

questions_with_duplicates = df_q.loc[df_q['qid'].isin(potential_queries_indices)] # -> queries for the retriever
df_chunks = df_q.loc[df_q['qid'].isin(all_ids)] # -> chunks to be indexed

1669
10000


In [ ]:
questions_with_duplicates.sample(5)

,qid,question
378507,311110,What are some interesting facts about the dolp...
86071,145197,Why do I ejaculate when I sleep?
112975,184740,What exactly is GOD?
84676,143132,What makes a woman get mad?
248965,362483,What do the meiosis phases look like?


In [ ]:
df_chunks.sample(5)

,qid,question
23607,44203,How true and accurate are exit polls?
82276,105503,What movie is the best movie of 2016?
191899,291383,How is your travel experience in Fly Emirates?
317084,442228,Will Puerto Rico become a US state soon?
64784,112527,How do you reset a Sony Ericsson phone?


### Evaluation functions

Define the relevant functions :
1. match index from questions to labels, by default, if the index pair iis not in the labelling table, consider that the the pair is not duplicates.
2. Compute the metrics
3. Evaluation loop over the relevant questions

In [ ]:
def hits(list_labels):
  # compute hit
  ## idea: if there is at least one hit, means list label contains at least one response that is similar to the query
  return 1 if any(list_labels) else 0

In [ ]:
def ap(list_labels):
  # compute average precision : https://www.evidentlyai.com/ranking-metrics/mean-average-precision-map

    # if not any(list_labels):
    if hits(list_labels) == 0: # in case the total number of relevant(denominator) is 0
        return 0
    l = np.array(list_labels)
    return sum([np.mean(l[:k+1]) * l[k]  for k in range(len(l))]) / sum(l)

In [ ]:
def compute_metrics(list_list_labels):
  hits_list = [hits(l) for l in list_list_labels]
  ap_list = [ap(l) for l in list_list_labels]
  return {'hit_rate': sum(hits_list)/len(hits_list), 'map': sum(ap_list)/len(ap_list)}

In [ ]:
# test_output = [0,0,0,0,0,0]
test_output = [1,0,0,1,1,0]

print(hits(test_output))#expects 1
print(ap(test_output)) #expects 0.7

1
0.7000000000000001


## Retriever implementation

### First Pipeline

In this first Pipeline we are going to use a simple sklearn pipeline for seamntic search.
#### Steps :
- TfIdf for vectorization
- [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) class for the search engine

In [ ]:
# define a TfidfVectorizer
tfidf_model = TfidfVectorizer()
tfidf_model.fit(df_chunks['question'].to_list())
# define a knn
vdb_tfidf = NearestNeighbors()
vdb_tfidf.fit(tfidf_model.transform(df_chunks['question'].to_list()))

def predict_duplicates_tfidf(query):
  # You can change it if you want
  query_vec = tfidf_model.transform([query])
  distances, indices = vdb_tfidf.kneighbors(query_vec)
  return df_chunks.iloc[indices[0]]

qq = questions_with_duplicates.iloc[100]
print('query : \n id: {} question: {}'.format(qq.qid, qq.question))
duplicate_q = df_chunks[df_chunks['qid']==df_labels.loc[(df_labels['is_duplicate']==1) & (df_labels['qid1']==qq.qid), 'qid2'].iloc[0]].iloc[0]
print('duplicate : \n id: {} question: {}'.format(duplicate_q.qid, duplicate_q.question))
predict_duplicates_tfidf(qq['question'])

query : 
 id: 42433 question: How do you evaluate Hillary Clinton to determine if she is fit to become the next U.S President?
duplicate : 
 id: 42434 question: Is Hillary clinton well enough to be president?


,qid,question
293773,242155,Whom do you expect to become the next presiden...
239166,42434,Is Hillary clinton well enough to be president?
266616,383818,Is there any chance that Hillary Clinton can s...
389496,149875,What will Hillary Clinton do if she does not w...
176736,137217,Who is going to be the next president of USA?


#### Evaluation loop

Sample of code, you can change it if you want.

In [ ]:
def get_label(qid, rqid, df_labels):
    result = df_labels['is_duplicate'][(df_labels['qid1'] == qid) & (df_labels['qid2'] == rqid)].to_list()
    return 0 if len(result) == 0 else result[0]

In [ ]:
all_labels = []
for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_tfidf(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.984, 'map': 0.9100944444444441}

### Second Pipeline : Sentence transformer pipeline

#### Steps:
 - Use the sentence_transformers package to load a model and embed the reviews
 - Since it is much more demanding than TfIdf, it could be necessary to subsample the dataset randomly
 - Build a pipeline with the same NearestNeighbors engine.

In [ ]:
# load sentence_transformers model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
# embed the reviews
embeddings = sbert_model.encode(df_chunks['question'].to_list())
# define a knn
vdb_sbert = NearestNeighbors()
vdb_sbert.fit(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NearestNeighbors()

In [ ]:
def predict_duplicates_sbert(query):
    #You may change this code
    query_vec = sbert_model.encode([query])
    distances, indices = vdb_sbert.kneighbors(query_vec)
    return df_chunks.iloc[indices[0]]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_sbert(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 1.0, 'map': 0.9619111111111113}

### Third Pipleine: BM25

After using distance based searchs methodfs, we are going to work with the BM25 algorithm. It was developped in late 70s but it is still very relevant. Some preprocessing of the text will be needed. A classical pipeline will be implemented.
#### Preprocessing

- Stopwords
- Stemming
- Transformer in lowercase

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def data_preprocessing(text):
    # define stop words and stemmer
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")
    # transformer text in lowercase and tokenize it
    tokens = word_tokenize(text.lower())
    # remove stop words and stemming
    cleaned_tokens = [stemmer.stem(tok) for tok in tokens if tok.isalnum() and tok not in stop_words]
    return cleaned_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
prepared_data = []
prepared_data = [data_preprocessing(item) for item in df_chunks['question'].to_list()]

#### BM25 engine

In [ ]:
# Create the BM25 Okapi engine
bm25_model = BM25Okapi(prepared_data)

### Queries

#### Preprocessing

In [ ]:
def query_preprocessing(query):
  # Preprocessing function for the different queries
  return data_preprocessing(query)

In [ ]:
def predict_duplicates_bm25(query):
    # You may change that
    preprocessed_query = query_preprocessing(query)
    best_matches = bm25_model.get_top_n(preprocessed_query, df_chunks.index, n=5)
    return df_chunks.loc[best_matches]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_bm25(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.98, 'map': 0.9162160317460314}

# Conclusion

Explain the results:
- how does the different methods compare ?
- What can be an explaination for the difference between the results of BM25 and TF-IDF veectors in a kNN?

**Answer** :
Among the three retrieval methods tested, the Sentence Transformer achieved the best performance with a hit rate of 1.0 and MAP of 0.96, significantly outperforming both BM25 and TF-IDF. This superior performance can be attributed to its ability to capture semantic relationships through pre-trained language models, allowing it to understand contextual meanings and synonyms rather than relying solely on lexical matching. However, this comes at the cost of computational efficiency, as evidenced by its slower processing speed of 19.08 iterations per second compared to the others.

The comparison between BM25 and TF-IDF reveals interesting insights into traditional retrieval methods. While TF-IDF showed slightly better in hit rate (0.984 vs. 0.98), BM25 map scores slightly higher (0.916 vs. 0.910). The reason for this small difference is that BM25 removes stop words and is more likely to match words with actual semantics, improving the ranking of relevant documents and thus improve map. In addition, stemming helps to match different forms of the same word and improves the generalization ability of the search. TF-IDF does not use stemming, e.g., when querying for “running” it will not be matched with “run” in a document, resulting in relevant documents being missed or ranked lower. TF-IDF, although simpler, has the highest computational efficiency, with 91.82 iterations per second, and is therefore a viable choice for applications where speed is critical and a slightly lower accuracy is acceptable.

# If you have more time (not evaluated):

If you have some spare time, you can:
1. Try to download and use a cross-encoder model
2. Try to use a HNSW k-NN from the FAISS library

- cross-encoder model

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def predict_duplicates_cross_encoder(query):
    top_ranks = cross_encoder.rank(query, df_chunks['question'].dropna().to_list())[:5]
    top_indices = [rank['corpus_id'] for rank in top_ranks]
    return df_chunks.iloc[top_indices]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_cross_encoder(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.99, 'map': 0.9352749999999997}

- HNSW k-NN

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.5 MB/s eta 0:00:00


In [ ]:
import faiss

# load sentence_transformers model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
# embed the reviews
embeddings = sbert_model.encode(df_chunks['question'].to_list())
# define a HNSW knn
index = faiss.IndexHNSWFlat(embeddings.shape[1], 32) # 32 is the number of neighbors
index.add(embeddings)

In [ ]:
def predict_duplicates_sbert_hnsw(query):
    query_vec = sbert_model.encode([query])
    distances, indices = index.search(query_vec, 5) # search for 5 nearnest nerghbors
    return df_chunks.iloc[indices[0]]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_sbert_hnsw(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.996, 'map': 0.9493888888888886}

For same sbert model, the running speed for HNSW KNN (\[00:07, 54.17it/s\]) is much faster then classic KNN (\[00:22, 19.08it/s\]) before.